In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

import pickle

from tensorflow.python.keras.models import save_model, load_model, Model
from tensorflow.python.keras.layers import Dense, Input, BatchNormalization, Concatenate
from tensorflow.python.keras.utils import plot_model
from tensorflow.python.keras.callbacks import History, Callback
from tensorflow.python.keras.optimizers import Adamax, Adam, Adagrad, SGD, RMSprop
from tensorflow.python.keras import backend as K

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

_v = '_v0'
fname = 'data.csv'
bname = 'data_blind.csv'
label_name = 'Label'

In [ ]:
data = pd.read_csv(fname)

In [ ]:
print(data.shape)
data.head()

In [ ]:
X = data.drop(columns=[label_name])
Y = data[label_name]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify=Y)
input_shape = x_train.shape[1]
labels = np.unique(Y)
class_weights = compute_class_weight("balanced", labels, Y)

In [ ]:
print("Input Shape", input_shape)
print("Labels", labels)
print("Class Weights", class_weights)

## Train

In [ ]:
activation = 'relu'
optimizer = 'adamax'
loss = 'binary_crossentropy'
metrics = ['binary_accuracy']
lr = 0.01
decay = 0.001
batch = 1000
epochs = 25
intializer = 'glorot_normal'

In [ ]:
def model_a(input, act, init):
    d1 = Dense(1000, activation=act, kernel_initializer=init, name='Model_A_1')(input)
    d2 = Dense(1000, activation=act, kernel_initializer=init, name='Model_A_2')(d1)
    d3 = Dense(1000, activation=act, kernel_initializer=init, name='Model_A_3')(d2)
    out = Dense(1, activation='sigmoid', kernel_initializer=init, name='Model_A_Output')(d3)
    return out

def model_b(input, act, init):
    d1 = Dense(100, activation=act, kernel_initializer=init, name='Model_B_1')(input)
    d2 = Dense(25, activation=act, kernel_initializer=init, name='Model_B_2')(d1)
    d3 = Dense(10, activation=act, kernel_initializer=init, name='Model_B_3')(d2)
    out = Dense(1, activation='sigmoid', kernel_initializer=init, name='Model_B_Output')(d3)
    return out

def model_c(input, act, init):
    d1 = Dense(1000, activation=act, kernel_initializer=init, name='Model_C_1')(input)
    d2 = Dense(500, activation=act, kernel_initializer=init, name='Model_C_2')(d1)
    d3 = Dense(250, activation=act, kernel_initializer=init, name='Model_C_3')(d2)
    d4 = Dense(100, activation=act, kernel_initializer=init, name='Model_C_4')(d3)
    out = Dense(1, activation='sigmoid', kernel_initializer=init, name='Model_C_Output')(d4)
    return out

def model_d(input, act, init):
    d1 = Dense(50, activation=act, kernel_initializer=init, name='Model_D_1')(input)
    d2 = Dense(50, activation=act, kernel_initializer=init, name='Model_D_2')(d1)
    d3 = Dense(50, activation=act, kernel_initializer=init, name='Model_D_3')(d2)
    out = Dense(1, activation='sigmoid', kernel_initializer=init, name='Model_D_Output')(d3)
    return out

def model_e(input, act, init):
    d1 = Dense(1000, activation=act, kernel_initializer=init, name='Model_E_1')(input)
    out = Dense(1, activation='sigmoid', kernel_initializer=init, name='Model_E_Output')(d1)
    return out

def model_f(input, act, init):
    d1 = Dense(50, activation=act, kernel_initializer=init, name='Model_F_1')(input)
    d2 = Dense(50, activation=act, kernel_initializer=init, name='Model_F_2')(d1)
    d3 = Dense(50, activation=act, kernel_initializer=init, name='Model_F_3')(d2)
    d4 = Dense(50, activation=act, kernel_initializer=init, name='Model_F_4')(d3)
    d5 = Dense(50, activation=act, kernel_initializer=init, name='Model_F_5')(d4)
    d6 = Dense(50, activation=act, kernel_initializer=init, name='Model_F_6')(d5)
    d7 = Dense(50, activation=act, kernel_initializer=init, name='Model_F_7')(d6)
    out = Dense(1, activation='sigmoid', kernel_initializer=init, name='Model_F_Output')(d7)
    return out

def model_g(input, act, init):
    d1 = Dense(100, activation='sigmoid', kernel_initializer=init, name='Model_G_1')(input)
    out = Dense(1, activation='sigmoid', kernel_initializer=init, name='Model_G_Output')(d1)
    return out


In [ ]:
def ensemble(input_shape, loss, metrics, activation = 'relu', optimizer = 'adamax', lr = 0.01, decay=0.001, initializer='glorot_normal'):
    
    inputs = Input(shape=(input_shape,), name='Input')
    norm = BatchNormalization(name='BatchNorm')(inputs)
    a = model_a(norm, activation, initializer)
    b = model_b(norm, activation, initializer)
    c = model_c(norm, activation, initializer)
    d = model_d(norm, activation, initializer)
    e = model_e(norm, activation, initializer)
    f = model_f(norm, activation, initializer)
    g = model_g(norm, activation, initializer)
    concat = Concatenate(name='Concat')([a, b, c, d, e, f, g])
    out = Dense(1, activation='sigmoid', kernel_initializer=initializer, name='Output')(concat)
    
    model = Model(inputs, out)
    
    if optimizer == 'adagrad':
        opt = Adagrad(lr = lr, decay = decay)
    elif optimizer == 'adam':
        opt = Adam(lr = lr, decay = decay)
    elif optimizer == 'adamax':
        opt = Adamax(lr = lr, decay = decay)
    elif optimizer == 'sgd':
        opt == SGD(lr = lr, decay = decay)
    elif optimzer == 'rmsprop':
        opt = RMSprop(lr = lr, decay = decay)
    else:
        opt = 'sgd'

    model.compile(loss=loss, optimizer=opt, metrics=metrics)
    
    return model

In [ ]:
model = ensemble(input_shape, loss=loss, metrics=metrics)

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes=True, show_layer_names = True, rankdir='TB', to_file='model' + _v + '.png')

In [ ]:
h = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=batch, epochs=epochs, verbose=1, shuffle=True, class_weight=class_weights)

In [ ]:
model.save('nn_classifier' + _v +'.h5')

In [ ]:
keys = h.history.keys()
for k in keys:
    print(k, h.history[k][-1])

In [ ]:
plt.plot(h.history[k[0]], label = [k[0]])
plt.plot(h.history[k[1]], label = [k[1]])
plt.legend()
plt.title("History")
plt.ylabel('metric')
plt.xlabel('Epoch')
plt.show()

In [ ]:
plt.plot(h.history[k[2]], label = [k[2]])
plt.plot(h.history[k[3]], label = [k[3]])
plt.legend()
plt.title("History")
plt.ylabel('metric')
plt.xlabel('Epoch')
plt.show()

## Test

In [ ]:
model = load_model('nn_classifier' + _v +'.h5')

In [ ]:
y_res = model.predict(x_test, verbose=1)

In [ ]:
threshold = np.arange(0.05, 0.95, 0.01)

In [ ]:
precision = [] # change metrics
recall = []
f1 = []
accuracy = []
for t in tqdm(threshold):
    Y_ = [0 if y_ < t else 1 for y_ in y_res]
    p, r, f, s = precision_recall_fscore_support(y_test, Y_)
    a = accuracy_score(y_test, Y_)
    
    precision.append(p[1])
    recall.append(r[1])
    accuracy.append(a)
    f1.append(f[1])


In [ ]:
idx = np.argwhere(f1 == np.max(f1)).flatten()
plt.figure(figsize=(10, 8))
plt.plot(threshold, recall, label='Recall')
plt.plot(threshold, precision, label='Precision')
plt.plot(threshold, f1, label='F1')
plt.plot(threshold, accuracy, label='Accuracy')
mx = 0
thresh = None
for i in idx:
    if f1[i] > mx: mx = f1[i]; thresh = round(threshold[i], 2)
    plt.plot(threshold[i], f1[i], 'ro')
    plt.text(threshold[i], f1[i], str(round(f1[i], 3)))
plt.title('Metrics')
plt.ylabel('Metric')
plt.xlabel('Threshold')
plt.legend()
plt.show()

In [ ]:
print("Threshold", thresh)

In [ ]:
y = [0 if _y < thresh else 1 for _y in y_res]

In [ ]:
acc = accuracy_score(y, y_test)
report = classification_report(y, y_test, digits=3)
cf = confusion_matrix(y, y_test)
a = auc(y, y_test)

In [ ]:
print(report)
print("Accuracy", acc)
print("AUC", a)

In [ ]:
sns.heatmap(cf, xticklabels=['No ' + label_name, label_name], yticklabels=['No ' + label_name, label_name], annot=True, fmt="d")
plt.title("Confusion matrix")
plt.ylabel('True Class')
plt.xlabel('Predicted class')
plt.show()

## Blind

In [ ]:
model = load_model('nn_classifier' + _v +'.h5')

In [ ]:
blind = pd.read_csv(bname)

In [ ]:
x_blind = blind.drop(columns=[label_name])
y_blind = blind[label_name]

In [ ]:
y_pred = model.predict(x_blind, verbose = 1)

In [ ]:
y_b = [0 if _y < thresh else 1 for _y in y_pred]

In [ ]:
acc = accuracy_score(y_b, y_blind)
report = classification_report(y_b, y_blind, digits=3)
cf = confusion_matrix(y_b, y_blind)
a = auc(y_b, y_blind)

In [ ]:
print(report)
print("Accuracy", acc)
print("AUC", a)

In [ ]:
sns.heatmap(cf, xticklabels=['No ' + label_name, label_name], yticklabels=['No ' + label_name, label_name], annot=True, fmt="d")
plt.title("Confusion matrix")
plt.ylabel('True Class')
plt.xlabel('Predicted class')
plt.show()